# Graph-RandomWalk 기반 개인화

참고. [cs246](http://web.stanford.edu/class/cs246/info.html)

## 용어
- bipartite graph: 그래프에서 성격이 다른 두개의 vertext 셋이 존재. 참고[Wikipedia](https://en.wikipedia.org/wiki/Bipartite_graph)
    - U: 사용자 히스토리에 관계된 vertex 셋
    - V: 위픽 세팅에 관계된 vertex 셋
    - U, V 의 intra-connection은 없음, inter-connection만 존재
- undirected edge between u and v: 사용자 히스토리의 딜 u와 위픽 세팅 v 간 유사도
    - f_u: 딜 u의 임베딩 벡터 (word2vec을 사용, unit-length in L2-norm)
    - f_v: 딜 v의 임베딩 벡터
    - s(u,v) = $\exp(\alpha \times f_u \cdot f_v)$ : 유사도
- P: u -> v로 전이할 확률
    - $\sum_{v} P_{uv} = 1$ : s(u,v)를 가공하여 생성 
- Q: v -> u로 전이할 확률
    - $\sum_{u} Q_{vu} = 1$ : s(u,v)를 가공하여 생성
    
## 방법
- v = (1,1, ..., 1) / |V| 로 초기화
- v' = v x Q x P 를 반복 (RANDOM WALK 수행)
    - 수렴할 때까지 (금새 수렴)
- 또는 QP 행렬의 first-left-eigenvector를 찾는다.

## 해석
- 최종 찾아진 v는 Page Rank와 유사하게, V 셋 내 deal의 중요도로 해석할 수 있다.
- 내림차순으로 정렬하여 serving!!!

In [1]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
from pymongo import MongoClient
import pymongo
import pandas as pd
import numpy as np
from operator import itemgetter

In [2]:
### twiceSpark1
es_url = '10.102.50.47:9200'
es = elasticsearch.Elasticsearch(es_url)
D = 100

## ES 관련 루틴


In [3]:
def es_search_dids_for_user(user_id, day_limit, gte_slot=1, ignore_consecutives=False):
    """
    user_id의 모든 v 가져오기
    day_limit 이전 것만 가져온다.
    return
    - 1st: v의 set
    - 2nd: 확장 정보 (v, rgtime, slot)
    """
    res = es.search(index='wepick_seq',
                  body={
                    "query": {
                      "bool": {
                        "must": {
                          "term": {"u": user_id}
                        },
                        "filter": {
                          "range": {
                            "rgtime": {
                              "lt": day_limit
                            }
                          }
                        }
                      }
                    },
                    "size": 64,
                    "sort": {"rgtime": "desc"}
                  }
                  )
    if res['hits']['total'] > 0:
        until_dt = pd.to_datetime(day_limit).to_pydatetime()
        filtered = []
        prev_v = None
        for hit in res['hits']['hits']:
            if ignore_consecutives == False or (prev_v is not None and prev_v != hit['_source']['v']):
                filtered.append((hit['_source']['v'], hit['_source']['rgtime'], hit['_source']['slot']))
            prev_v = hit['_source']['v']
        return set(map(lambda x: x[0], filtered)), filtered
    return None, None

In [4]:
def es_gather_word2vec_raw(dids):
    """
    dids로부터, word2vec을 모은다.
    return
    - dids: unit-length w2v (normalized by L2-norm)
    """
    res = es.search(index='deal_word2vec', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["values", "v"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    dic = {}
    zerovec = np.zeros((D,), dtype=np.float32)
    for hit in res['hits']['hits']:
        did = hit['_source']['v']
        vec = np.array(hit['_source']['values'])
        if len(vec) > 0 and np.allclose(vec, zerovec) == False:
            vec /= np.sqrt(np.sum(vec**2))
            dic[did] = vec
    return dic


In [5]:
def es_read_wepick_setting(dt, start_slot=20):
    """
    위픽 세팅 로딩
    """
    res = es.search(index='wepick_setting_ext', 
                body={
                    'query': {
                        'term': {'dt': dt }
                        }                        
                    }
               )
    if res['hits']['total'] > 0:
        dic = {}
        vec = []
        for s in res['hits']['hits'][0]['_source']['settings']:
            if s['slot'] >= start_slot:
                dic[s['slot']] = s['did']
                vec.append(s['did'])
        return vec, dic
    return None, None

In [6]:
def es_scan_extra_by_dids(dids):
    """
    dids로부터, mn, tn1를 가져온다.
    """
    res = es.search(index='dealinfos', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["mn", "tn1", "did"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    dic = {}
    for hit in res['hits']['hits']:
        dic[hit['_source']['did']] = (hit['_source']['mn'], hit['_source']['tn1'])
    return dic

### mongoDB for ActionInfos2

In [7]:
client = MongoClient(host='35.190.239.204', port=27017, username='praha_read', password='praha!@#', authSource='praha')

db = client['praha']

col = db['memberActionInfos2']

In [8]:
def mg_get_ordered_dids(mid, lt_day="20180411", limit=32):
    """
    구매한 딜들을 조회
    """
    result = col.find({"mid":mid, 'ft.o':{"$ne":[]}, 'day':{"$lt":lt_day}}, {'day':1, 'ft.o': 1, '_id':0}).sort('day', pymongo.DESCENDING).limit(limit)
    out = set()
    for res in result:
        out.update(list(map(lambda x: x['did'], res['ft']['o'])))
    return out

In [9]:
def mg_get_clicked_dids(mid, lt_day="20180411", limit=32, use_search_induced_click=False):
    """
    클릭한 딜들을 조회
    """
    result = col.find({"mid":mid, 'ft.c':{"$ne":[]}, 'day':{"$lt":lt_day}}, {'day':1, 'ft.c': 1, '_id':0}).sort('day', pymongo.DESCENDING).limit(limit)
    out = set()
    for res in result:
        out.update(list(map(lambda x: x['did'], 
                            res['ft']['c'] if use_search_induced_click == False else filter(lambda x: x['s'] != '', res['ft']['c'])
                           )))
    return out

## Ranking 관련

In [10]:
def get_refined_scores(scores, extra_dic):
    refined_scores = []
    for did, score in scores:
        if did in extra_dic:
            refined_scores.append((score, did, extra_dic[did][0], extra_dic[did][1]))
        else:
            refined_scores.append((score, did, "", ""))
    return refined_scores

In [11]:
def print_result(out, wepick_slot_dic):
    for s, did, title, cate in out:
        org_slot = wepick_slot_dic[did] if did in wepick_slot_dic else -1
        print((s, did, title, org_slot, cate))
            

## Random Walk 관련

In [12]:
def get_w2vec_matrix(A):
    """
    A = dict: dealidx -> np.array((D,), dtype=np.float32)
    """
    
    deal_to_idx =  {k:i for i, k in enumerate(A.keys())}
    idx_to_deal =  {i:k for i, k in enumerate(A.keys())}
    
    M = np.array(list(A.values()), dtype=np.float32)
    return M, idx_to_deal

In [13]:
def get_random_walk_matrix(U, V, alpha=5.0):
    """
    V->U->V 로 전이시켜주는 확률 매트릭스를 리턴한다고 생각하자.
    """
    P_ = np.matmul(U, V.T)
    Q_ = np.matmul(V, U.T)
    
    P = np.exp(alpha*P_)
    Q = np.exp(alpha*Q_)

    P = P / np.sum(P, axis=1, keepdims=True)
    Q = Q / np.sum(Q, axis=1, keepdims=True)
    
    return np.matmul(Q, P)    

In [14]:
def do_random_walk(QQ, prior_v=None, tol=1e-8):
    """
    QQ: |V| x |V| matrix
    tol 미만 될 때까지 반복
    시작: uniform probability가 되게 설정
    """
    y = np.ones((1, QQ.shape[0]), dtype=np.float32) / QQ.shape[0] if prior_v is None else prior_v
    while True:
        new_y = np.dot(y, QQ)
        if np.linalg.norm(y - new_y) < tol:
            return new_y.squeeze()
        y = new_y   

In [15]:
def calc_score_by_random_walk(W, prior_v=None):
    y = do_random_walk(W, prior_v)
    sorted_ii = np.argsort(y)[::-1]
    scores = list(map(lambda x: (jj[x], y[x]), sorted_ii))
    return scores

## BiRank 알고리즘 (확장)
- 참고 [논문링크](https://arxiv.org/pdf/1708.04396.pdf)
    - 4페이지 참고
- 내용
    - [HITS 알고리즘 변형] (https://en.wikipedia.org/wiki/HITS_algorithm)
    - u, v 를 번갈아며 RandomWalk

In [16]:
def get_W_for_birank(U, V, scale=5.0):
    return np.exp(scale * np.matmul(U, V.T))    

In [17]:
def get_prior_for_birank(W):
    """
    prior 값 계산 (베이직)
    """
    n_u, n_v = W.shape[0], W.shape[1]
    u_0 = np.ones((n_u,), dtype=np.float32) / n_u
    v_0 = np.ones((n_v,), dtype=np.float32) / n_v

    return u_0, v_0

In [18]:
def iterative_birank_algorithm(W, alpha=0.95, beta=0.95, tol=1e-5):
    Du = np.sum(W, axis=1)**-(0.5)
    Dv = np.sum(W, axis=0)**-(0.5)
    S = np.matmul(np.matmul(np.diag(Du), W), np.diag(Dv))
    

    u_0, v_0 = get_prior_for_birank(W)
    
    n_u, n_v = W.shape[0], W.shape[1]
    u = np.ones((n_u,), dtype=np.float32) / n_u
    v = np.ones((n_v,), dtype=np.float32) / n_v

    tol = 1e-4
    while True:
        new_v = alpha * np.dot(S.T, u) + (1-alpha)*v_0
        new_u = beta * np.dot(S, v) + (1-beta)*u_0
        if np.linalg.norm(v - new_v) < tol and np.linalg.norm(u - new_u) < tol:
            break
        v = new_v
        u = new_u
    return u, v

In [19]:
def calc_score_by_birank_algorithm(W, prior_v=None):
    u, v = iterative_birank_algorithm(W)
    sorted_ii = np.argsort(v)[::-1]
    scores = list(map(lambda x: (jj[x], v[x]), sorted_ii))
    return scores

### Wepick Setting load

In [20]:
# 2018-04-11 21 시의 위픽 세팅 로딩
wepick_setting, wepick_dic = es_read_wepick_setting('2018-04-11 21')

wepick_slot_dic = dict(zip(wepick_dic.values(), wepick_dic.keys()))

### deal_profile loading

In [21]:
# 위픽 세팅에 따른 딜들에 대한 deal_profile을 생성
deal_profile_dic = es_gather_word2vec_raw(wepick_setting)
extra_dic = es_scan_extra_by_dids(wepick_setting)

## 3월 11 -  4월 10일까지 위픽 클릭 데이터에 대해 구성한 user_profile에 대한 테스트

In [22]:
deals_user_viewed, ex = es_search_dids_for_user(1000007, '2018-04-11', ignore_consecutives=False)

user_profile_dic = es_gather_word2vec_raw(list(deals_user_viewed))

In [23]:
U, ii = get_w2vec_matrix(user_profile_dic)
V, jj = get_w2vec_matrix(deal_profile_dic)

In [24]:
W = get_random_walk_matrix(U, V)

In [25]:
out = get_refined_scores(calc_score_by_random_walk(W), extra_dic)

In [26]:
print_result(out, wepick_slot_dic)

(0.054981604, 3515524, '[무료배송] 롱티/티셔츠/원피스', 45, '티셔츠')
(0.044169676, 3512593, '[무료배송] 봄 아동복 브랜드 연합전', 36, '아동공용의류')
(0.043535206, 3527477, '[투데이특가] 니트/가디건/원피스 외', 61, '원피스')
(0.0417196, 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', 55, '티셔츠')
(0.038509108, 3525317, '[무료배송] 빅사이즈/원피스/롱티', 28, '원피스')
(0.038321517, 3525500, '[하객패션] 포커스 봄구성완벽해', 74, '티셔츠')
(0.038265932, 3541064, '[위메프] 10만 포인트+5,000P', 33, '온라인 이용권')
(0.038145103, 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', 44, '티셔츠')
(0.03312627, 3514459, '[심야특가] 파파야 여성 의류 모음전', 32, '티셔츠')
(0.02731631, 3512215, '[6천원쿠폰] 기습쿠폰전 오늘마지막!', 30, '색조메이크업')
(0.025446719, 3513766, '[어린이날] 해피버스 7부/자가드내의', 75, '내의/잠옷/속옷')
(0.024799671, 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', 66, '아동공용의류')
(0.023921728, 3492158, '[할인사건] 홍콩VS마카오 항공&자유', 77, '홍콩')
(0.023306064, 3527053, '[투데이특가] 더사랑이 여름 아동복', 43, '아동공용의류')
(0.019576777, 3486081, '사이판 월드 골드 4/5일+마나가하', 22, '사이판')
(0.013502407, 1438471, '[20%쿠폰] 니베아 립밤 바디로션 ', 34, '립케어')
(0.013376844, 3528363, '[하객패션] 백화점 잡화 267종! +20%', 27, '벨트')
(0.013

### birank 알고리즘 결과 (참고)

In [27]:
out = get_refined_scores(calc_score_by_birank_algorithm(get_W_for_birank(U, V)), extra_dic)

In [28]:
print_result(out, wepick_slot_dic)

(0.032587953, 3515524, '[무료배송] 롱티/티셔츠/원피스', 45, '티셔츠')
(0.029440157, 3512593, '[무료배송] 봄 아동복 브랜드 연합전', 36, '아동공용의류')
(0.02903204, 3527477, '[투데이특가] 니트/가디건/원피스 외', 61, '원피스')
(0.028433582, 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', 55, '티셔츠')
(0.02816983, 3541064, '[위메프] 10만 포인트+5,000P', 33, '온라인 이용권')
(0.027414763, 3525317, '[무료배송] 빅사이즈/원피스/롱티', 28, '원피스')
(0.02741185, 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', 44, '티셔츠')
(0.027398152, 3525500, '[하객패션] 포커스 봄구성완벽해', 74, '티셔츠')
(0.025463287, 3514459, '[심야특가] 파파야 여성 의류 모음전', 32, '티셔츠')
(0.024016457, 3512215, '[6천원쿠폰] 기습쿠폰전 오늘마지막!', 30, '색조메이크업')
(0.023033839, 3513766, '[어린이날] 해피버스 7부/자가드내의', 75, '내의/잠옷/속옷')
(0.022712091, 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', 66, '아동공용의류')
(0.02268634, 3492158, '[할인사건] 홍콩VS마카오 항공&자유', 77, '홍콩')
(0.02188199, 3527053, '[투데이특가] 더사랑이 여름 아동복', 43, '아동공용의류')
(0.020620363, 3486081, '사이판 월드 골드 4/5일+마나가하', 22, '사이판')
(0.017246699, 1438471, '[20%쿠폰] 니베아 립밤 바디로션 ', 34, '립케어')
(0.016858192, 3515873, '[투데이특가] 레이스 덧신 1+1+1', 65, '스타킹/양말')
(0.0168

## 구매 did로 부터 랭킹 테스트

- 구매 did 들의 word2vec을 사용

In [29]:
deals_user_purchased = mg_get_ordered_dids(1000007, limit=32)
user_profile_dic = es_gather_word2vec_raw(list(deals_user_purchased))

In [30]:
U, ii = get_w2vec_matrix(user_profile_dic)
V, jj = get_w2vec_matrix(deal_profile_dic)

In [31]:
W = get_random_walk_matrix(U, V)

In [32]:
out = get_refined_scores(calc_score_by_random_walk(W), extra_dic)

In [33]:
print_result(out, wepick_slot_dic)

(0.04071803, 3513766, '[어린이날] 해피버스 7부/자가드내의', 75, '내의/잠옷/속옷')
(0.034216903, 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', 66, '아동공용의류')
(0.028112475, 3529131, '휴비딕 체온계 & 탕온도 특가전', 24, '유아건강/유아위생용품')
(0.027766462, 3527053, '[투데이특가] 더사랑이 여름 아동복', 43, '아동공용의류')
(0.025906578, 3500355, '[투데이특가] 좋은느낌 생리대 38+38', 47, '화이트')
(0.024456864, 3532552, '[투데이특가] 맑음 배도라지 50팩', 86, '아기간식/아기음료')
(0.01846739, 3464309, '[쿠폰할인] LG 공기청정기 AS181DAW', 57, '공기청정기')
(0.018372865, 3522395, '[롯데] 르까프 아동/성인 빅세일', 46, '남성 티셔츠/상의 기타')
(0.017763102, 3519047, '[투데이특가] 풀무원 간편국15+5입', 31, '즉석밥/국/카레')
(0.016893266, 3527569, '[투데이특가] 아디다스 그래픽스케일', 91, '반팔 티셔츠')
(0.01687536, 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', 41, '유아퍼즐')
(0.016298974, 3515873, '[투데이특가] 레이스 덧신 1+1+1', 65, '스타킹/양말')
(0.016289897, 3544419, '[게릴라특가] 봄맞이 카페트 150x200', 71, '카페트/러그')
(0.016093565, 3518630, '[투데이특가] 꺾어먹는 비요뜨 12개', 79, '두유/우유')
(0.015665738, 1438471, '[20%쿠폰] 니베아 립밤 바디로션 ', 34, '립케어')
(0.015557776, 3512215, '[6천원쿠폰] 기습쿠폰전 오늘마지막!', 30, '색조메이크업')
(0.0151923075, 35041

## 클릭 did로 부터 랭킹 테스트

- 유저가 클릭한 did 들의 word2vec을 사용
- use_search_induced_click==True 면, 검색후 클릭된 did만 가져온다.

In [34]:
deals_user_search_click= mg_get_clicked_dids(1000007, limit=32, use_search_induced_click=True)
user_profile_dic = es_gather_word2vec_raw(list(deals_user_search_click))

In [35]:
U, ii = get_w2vec_matrix(user_profile_dic)
V, jj = get_w2vec_matrix(deal_profile_dic)

In [36]:
W = get_random_walk_matrix(U, V)

In [37]:
out = get_refined_scores(calc_score_by_random_walk(W), extra_dic)

In [38]:
print_result(out, wepick_slot_dic)

(0.037897825, 3541064, '[위메프] 10만 포인트+5,000P', 33, '온라인 이용권')
(0.03615521, 3521785, '[쿠폰할인] 중고폰 노트5/S7/엣지', 51, '공기계-미사용/미개봉')
(0.026868312, 3504137, '[리빙위크] 비즈니스보루네오 소파', 78, '소파')
(0.025997974, 3512215, '[6천원쿠폰] 기습쿠폰전 오늘마지막!', 30, '색조메이크업')
(0.025360812, 3464309, '[쿠폰할인] LG 공기청정기 AS181DAW', 57, '공기청정기')
(0.02296534, 3544419, '[게릴라특가] 봄맞이 카페트 150x200', 71, '카페트/러그')
(0.021816049, 3527569, '[투데이특가] 아디다스 그래픽스케일', 91, '반팔 티셔츠')
(0.020876907, 3511172, '[컬러풀] GTX1060 6GB 그래픽카드', 80, '그래픽카드')
(0.02074184, 3061867, '[추가쿠폰] 삼성 냉장고 RH81K8050SA', 50, '양문형 냉장고')
(0.020123716, 3507588, '[투데이특가] 보노바스켓 정리함 80L', 87, '리빙박스/수납함')
(0.019867865, 3509649, '[리빙위크] 리빙숲 리빙박스72L 4개', 49, '리빙박스/수납함')
(0.01908769, 1438471, '[20%쿠폰] 니베아 립밤 바디로션 ', 34, '립케어')
(0.018132832, 3512421, '[투데이특가] 휴대용 핸디 선풍기!', 85, '휴대용 선풍기')
(0.017961042, 3521723, '[투데이특가] 샤오미 공기청정기', 25, '공기청정기')
(0.017633663, 2258611, '[사은품증정] 리빙웰 에어프라이어', 84, '튀김기')
(0.016981699, 3068897, '[가전쿠폰] LG 13kg 통돌이세탁기', 37, '일반 세탁기')
(0.016872227, 311925